# Dependencies

In [ ]:
!pip install psycopg2
!pip install gensim==3.6.0

# Libraries

In [6]:
import pandas as pd
import numpy as np
import psycopg2 as postgres
import psycopg2.extras
import math
from gensim.models import Word2Vec
from tqdm import tqdm
from sshtunnel import SSHTunnelForwarder

# Functions

In [7]:
def connect():
    """
    Establishes a connection to a PostgreSQL database.

    Returns:
        psycopg2.extensions.connection: A connection object to the database.
    """
    conn = None
    try:
        conn = psycopg2.connect(
            host="localhost",
            database="austin_test",  # Database name
            user="postgres",         # Database username
            password="root"          # Database password
        )
    except psycopg2.Error as e:
        print(e)
    return conn

In [8]:
def closeConnection(conn):
    """
    Closes the connection to a PostgreSQL database.

    Args:
        conn (psycopg2.extensions.connection): A connection object to the database.

    Returns:
        bool: True if the connection was successfully closed, False otherwise.
    """
    success = False
    try:
        conn.close()
        success = True
    except psycopg2.Error as e:
        print(e)
    
    return success

In [9]:
def executeQuery(conn, sql):
    """
    Executes a SQL query on a PostgreSQL database.

    Args:
        conn (psycopg2.extensions.connection): A connection object to the database.
        sql (str): The SQL query to execute.

    Returns:
        list: A list of records (tuples) retrieved from the database.
    """
    record = None
    try:
        cur = conn.cursor()
        cur.execute(sql)
        record = cur.fetchall()
        cur.close()
    except psycopg2.Error as e:
        print(e)
        cur.execute("ROLLBACK")
        cur.close()
    return record

In [10]:
def executeInsert(conn, sql):
    """
    Executes an SQL INSERT statement on a PostgreSQL database.

    Args:
        conn (psycopg2.extensions.connection): A connection object to the database.
        sql (str): The SQL INSERT statement to execute.

    Returns:
        bool: True if the insertion was successful, False otherwise.
    """
    success = False
    try:
        cur = conn.cursor()
        cur.execute(sql)
        conn.commit()
        cur.close()
        success = True
    except psycopg2.Error as e:
        print(e)
        cur.execute("ROLLBACK")
        cur.close()

    return success

In [11]:
def getPOIInformation(conn, business_id):
    """
    Retrieves information about a Point of Interest (POI) based on its ID.

    Args:
        conn (psycopg2.extensions.connection): A connection object to the database.
        business_id (str): The unique ID of the POI.

    Returns:
        list: A list of tuples containing checkin count and name for the specified POI.
    """
     sql = """
        SELECT checkin_count, name FROM pois_information WHERE id  = \'"""+str(business_id)+ """\'
    ;"""

    result = executeQuery(conn, sql)
    return result

In [12]:
# Find all points in the bin centered around a POI, along with their information (categories and check-in counts)
def getBinPOIsInformation(conn, business_id, bin_number):
    """
    Retrieves information about all points within a specific bin centered around a given POI.

    Args:
        conn (psycopg2.extensions.connection): Database connection.
        business_id (str): ID of the central POI.
        bin_number (int): Bin number.

    Returns:
        list: List of dictionaries containing information for each point in the bin.
            Each dictionary includes keys: 'fk_poi_id_context', 'name', 'level', 'checkin_count', and 'distance_m'.
    """
    result = None

    sql = """
        SELECT fk_poi_id_context, name, level, checkin_count, distance_m 
        FROM bins_pois_information 
        WHERE fk_poi_id_center = \'"""+str(business_id)+"""\' AND fk_bin_number = """+str(bin_number)+""";"""

    result = executeQuery(conn, sql)

    return result


In [13]:
# Function that retrieves information from a materialized view related to OSM
def getBinOSMInformation(conn, business_id, bin_number, materialized_view):
    """
    Retrieves information from a materialized view related to OpenStreetMap (OSM).

    Args:
        conn (psycopg2.extensions.connection): Database connection.
        business_id (str): ID of the central POI.
        bin_number (int): Bin number.
        materialized_view (str): Name of the materialized view.

    Returns:
        list: List of dictionaries containing information for the specified POI and bin.
            Each dictionary includes relevant keys from the materialized view.
    """
    result = None

    sql = """
        SELECT *
        FROM """+materialized_view+"""
        WHERE id = \'"""+str(business_id)+"""\' AND number = """+str(bin_number)+""";"""

    # print (sql)

    result = executeQuery(conn, sql)

    return result

## GeoContext2Vec

In [17]:
import math
from tqdm import tqdm

def calculateBinOSMPolygon_Disco(df, bin_number, mi=20):
    """
    Generates binary relations between types of POIs and the polygonal geographic features. The binary relations are directly saved in the disco

    Args:
        df (pandas.DataFrame): DataFrame containing POI information.
        bin_number (int): related to context radius.
        mi (int, optional): Value for mi. Defaults to 20.

    Returns:
        None
    """

    #Occurrence proportion (OP) and Space proportion (SP) weight
    weights = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
    radius = (1+bin_number)*100
    print("executing radius:", radius, "m")

    # File to save directly to disk
    writers = []
    csv_files = []
    for w in weights:
        file_name = './geographic/GEOC2VECpiR/austin-sl-tuple-geoc2vec-' + str(bin_number) + 'bins_polygons_information-wgt' + str(w) + 'pfpe-c.csv'
        csv_file = open(file_name, "w", newline='')
        writer = csv.writer(csv_file, delimiter=',')
        writer.writerow(["poi_id_center",
                         "center_poi",
                         "center_poi_level",
                         "context_osm"])

        csv_files.append(csv_file)
        writers.append(writer)

        # Creating communication channel with the database
        try:
            with SSHTunnelForwarder(
                ('localhost', 22),
                ssh_username="root",
                ssh_password="root",
                remote_bind_address=('localhost', 5432)
            ) as server:
                server.start()
    
                params = {'database': 'austin_test',
                          'user': 'postgres',
                          'password': 'root',
                          'host': 'localhost',
                          'port': server.local_bind_port
                          }
    
                connection = psycopg2.connect(**params)
    
                for id_01, poi in tqdm(df.iterrows(), total=df.shape[0]):
                    # [business_id, checkin, category]
                    poi_information = getPOIInformation(connection, poi['business_id'])
                
                    # [business_id, checkin, category, distance_m]
                    if bin_number == 0:
                        bin_osm_information = getBinOSMInformation(connection, poi['business_id'], bin_number, 'bins_polygons_information')
                        bin_osm_building_information = getBinOSMInformation(connection, poi['business_id'], bin_number, 'bins_polygons_building_information')
                    else:
                        bin_osm_information = getBinOSMInformation(connection, poi['business_id'], bin_number, 'continuous_bins_polygons_information')
                        bin_osm_building_information = getBinOSMInformation(connection, poi['business_id'], bin_number, 'continuous_bins_polygons_building_information')
                
                    # Calculating the two parameters below
                    # oc - total number of different polygons in the bin
                    oc = 0
                    sc = 0
                
                    # If the bin contains any information
                    if len(bin_osm_information) > 0:
                        tags = list(dict(bin_osm_information[0]).keys())
                        bin_osm_information = pd.DataFrame(bin_osm_information, columns=tags)
                
                        # Checking how many different polygon types exist
                        oc = oc + bin_osm_information.iloc[:, 2:len(tags) - 4][~bin_osm_information.iloc[:, 2:len(tags) - 4].isin(['None'])].count().sum()
                        sc = sc + (bin_osm_information.iloc[:,2:len(tags)-4][~bin_osm_information.iloc[:,2:len(tags)-3].isin(['None'])].count(axis=1)*bin_osm_information['way_area_in']).sum()
                        
                        #Excluding ids and bin_number
                        tags = tags[2:len(tags)-4]

                    # If the bin contains any information
                    if len(bin_osm_building_information) > 0:
                        tags_buildings = list(dict(bin_osm_building_information[0]).keys())
                        bin_osm_building_information = pd.DataFrame(bin_osm_building_information, columns=tags_buildings)
                    
                        oc = oc + bin_osm_building_information.iloc[0]['building_count']
                        sc = sc + bin_osm_building_information.iloc[0]['area_total']
                    
                        # Excluding IDs and bin_number
                        tags_buildings = tags_buildings[1:2]
                    
                    # Additions are based on labels
                    
                    # Checking area completeness
                    empty_area = -1
                    if sc < area:
                        oc += 1
                        sc += area - sc
                        empty_area = area - sc
                    
                    # To avoid division by zero
                    if oc != 0:
                        if (len(bin_osm_information) > 0):
                            for tag in tags:
                                
                                #Iterate through each tag that indicates a geographic feature.
                                geographic_features = set(bin_osm_information[tag].values)

                                for feature in geographic_features:

                                    if(feature != None):

                                        #sf = all area o tag
                                        #op = all occurences of tag 
                                        
                                        sf = bin_osm_information[bin_osm_information[tag] == feature]['way_area_in'].sum()
                                        of = bin_osm_information[bin_osm_information[tag] == feature][tag].count()

                                        SP = math.ceil((sf/sc)*mi)        
                                        OP = math.ceil((of/oc)*mi)
                                        
                                        for idx, w in enumerate(weights):
                                            
                                            aug = int(math.ceil((w*SP) + ((1 - w)*OP)))

                                            if (aug <= 0):
                                                aug = 1

                                            name = "polygons_"+tag+"_"+feature

                                            for center_poi in poi_information: # for each POI type tki
                                                #Replicating binary relation <poi type, geographic feature>
                                                for b in range(aug):

                                                    line = [str(poi['business_id']), 
                                                            str(center_poi['name']),
                                                            str(center_poi['level']),
                                                            str(name)]
                                                    writers[idx].writerow(line)

                        #Calculating co-occurrence with polygons that are exclusively buildings.                   
                        if (len(bin_osm_building_information) > 0):

                            for id_02, row in bin_osm_building_information.iterrows():

                                    #sf = all area o tag
                                    #of = all occurences of tag

                                    sf = row['area_total']
                                    of = row['building_count']

                                    SP = math.ceil((sf/sc)*mi)        
                                    OP = math.ceil((of/oc)*mi)
                                    
                                    for idx, w in enumerate(weights):

                                        aug = int(math.ceil((w*SP) + ((1 - w)*OP)))
                                        if (aug <= 0):
                                            aug = 1

                                        name = 'polygons_building_yes'

                                        for center_poi in poi_information: # for each POI type tki
                                            #Replicating binary relation <poi type, geographic feature>
                                            for b in range(aug):

                                                line = [str(poi['business_id']), 
                                                        str(center_poi['name']),
                                                        str(center_poi['level']),
                                                        str(name)]
                                                writers[idx].writerow(line)

                            
                for csv_file in csv_files:
                    csv_file.close()
                connection.close()
    
    except Exception as e:
        print(e)
    
    return None

In [18]:
import csv
import math
from tqdm import tqdm
def calculateBinOSMPolygon_distance_Disk(df, bin_number, mi = 20):

    """
    Generates binary relations between types of POIs and the polygonal geographic features using distance to penalize the relations. 
    The binary relations are directly saved in the disk.

    Args:
        df (pandas.DataFrame): DataFrame containing POI information.
        bin_number (int): related to context radius.
        mi (int, optional): Value for mi. Defaults to 20.

    Returns:
        None
    """
    #Occurrence proportion (OP) and Space proportion (SP) weight
    weights = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
    radius = (1+bin_number)*100
    
    print("executing radius:", radius, "m")

    # File to save directly to disk
    writers = []
    csv_files = []
    for w in weights:
        
    
        file_name = './austin-sl-tuple-geoc2vec-' + str(bin_number) + 'bins_polygons_information-wgt'+str(w)+'pfp-c.csv'
    
    
        csv_file = open(file_name, "w", newline='')
        writer = csv.writer(csv_file, delimiter=',')
        writer.writerow(["poi_id_center",
                         "center_poi",
                         "center_poi_level",
                         "context_osm"])
        
        csv_files.append(csv_file)
        writers.append(writer)

    #Creating communication channel with postgres database
    try:

        with SSHTunnelForwarder(
            ('localhost', 22),
            ssh_username="root",
            ssh_password="root", 
            remote_bind_address=('localhost', 5432)) as server:

                server.start()

                params = {'database': 'austin_test',
                       'user': 'postgres',
                       'password': 'root',
                       'host': 'localhost',
                       'port': server.local_bind_port
                }

                connection = psycopg2.connect(**params)

                for id_01, poi in tqdm(df.iterrows(), total=df.shape[0]):

                    #[business_id, checkin, category]
                    poi_information = getPOIInformation(connection, poi['business_id'])

                    #[business_id, checkin, category, distance_m]
                    if (bin_number == 0):
                        bin_osm_information = getBinOSMInformation(connection, poi['business_id'], bin_number, 'bins_polygons_information')
                        bin_osm_building_information = getBinOSMInformation(connection, poi['business_id'], bin_number, 'bins_polygons_building_information')

                    else:
                        bin_osm_information = getBinOSMInformation(connection, poi['business_id'], bin_number, 'continuous_bins_polygons_information')
                        bin_osm_building_information = getBinOSMInformation(connection, poi['business_id'], bin_number, 'continuous_bins_polygons_building_information')


                    #Calculating oc and sc from the algorithm
                    oc = 0
                    sc = 0

                    #if the context contains some information
                    if (len(bin_osm_information) > 0):
                        tags = list(dict(bin_osm_information[0]).keys())
                        bin_osm_information = pd.DataFrame(bin_osm_information, columns = tags)

                        #Verifying how many geagraphif features (from polygons) exists
                        oc = oc + bin_osm_information.iloc[:,2:len(tags)-4][~bin_osm_information.iloc[:,2:len(tags)-4].isin(['None'])].count().sum()
                        
                        sc = sc + (bin_osm_information.iloc[:,2:len(tags)-4][~bin_osm_information.iloc[:,2:len(tags)-3].isin(['None'])].count(axis=1)*bin_osm_information['way_area_in']).sum()
                        
                        #Excluding ids and bin_number
                        tags = tags[2:len(tags)-4]

                    #if the context contains some information (for building polygons)
                    if (len(bin_osm_building_information) > 0):
                        tags_buildings = list(dict(bin_osm_building_information[0]).keys())
                        bin_osm_building_information = pd.DataFrame(bin_osm_building_information, columns = tags_buildings)

                        oc = oc + bin_osm_building_information.iloc[0]['building_count']
                        
                        sc = sc + bin_osm_building_information.iloc[0]['area_total']

                        #Excluding ids and bin_number
                        tags_buildings = tags_buildings[1:2]

                    #to avoid zero division
                    if(oc != 0):
                        if (len(bin_osm_information) > 0):
                        
                            for tag in tags:
                                #Iterate over each geographic feature
                                geographic_features = set(bin_osm_information[tag].values)

                                for feature in geographic_features:

                                    if(feature != None):

                                        #sf = all area o tag
                                        #op = all occurences of tag 

                                        sf = bin_osm_information[bin_osm_information[tag] == feature]['way_area_in'].sum()
                                        of = bin_osm_information[bin_osm_information[tag] == feature][tag].count()
                                        dst = bin_osm_information[bin_osm_information[tag] == feature]['distance_m'].mean()


                                        SP = math.ceil((sf/sc)*mi)        
                                        OP = math.ceil((of/oc)*mi)

                                        for idx, w in enumerate(weights):

                                            #Calculating relative distance
                                            dst_rel = (dst/radius)
                                            aug = int(math.ceil(((w*SP) + ((1 - w)*OP))/(1+dst_rel)))
                                            if (aug <= 0):
                                                aug = 1

                                            name = "polygons_"+tag+"_"+feature

                                            for center_poi in poi_information: # Para cada tki
                                                #Replicating binary relations using aug information
                                                for b in range(aug):

                                                    line = [str(poi['business_id']), 
                                                            str(center_poi['name']),
                                                            str(center_poi['level']),
                                                            str(name)]
                                                    writers[idx].writerow(line)

                        #Calculating the co-occurence with polygons related to buidlings                    
                        if (len(bin_osm_building_information) > 0):

                            for id_02, row in bin_osm_building_information.iterrows(): 

                                #sf = all area o tag
                                #of = all occurences of tag

                                sf = row['area_total']
                                of = row['building_count']
                                
                                if(bin_number == 0):
                                    dst=50
                                else:
                                    dst = row['building_avg_distance']

                                SP = math.ceil((sf/sc)*mi)        
                                OP = math.ceil((of/oc)*mi)

                                for idx, w in enumerate(weights):

                                    dst_rel = (dst/radius)
                                    aug = int(math.ceil(((w*SP) + ((1 - w)*OP))/(1+dst_rel)))
                                    if (aug <= 0):
                                        aug = 1

                                    name = 'polygons_building_yes'

                                    for center_poi in poi_information: # Para cada tki
                                        #Aumentando-o pelo fator b
                                        for b in range(aug):

                                            line = [str(poi['business_id']), 
                                                    str(center_poi['name']),
                                                    str(center_poi['level']),
                                                    str(name)]
                                            writers[idx].writerow(line)
                        
                            
                for csv_file in csv_files:
                    csv_file.close()
                connection.close()
    
    except Exception as e:
        print(e)

    
    return None

In [19]:
# Generates binary relations between types of POIs and the polygonal geographic features using distance to penalize the relations.
# The binary relations are directly saved in the disk.
import csv
import math
from tqdm import tqdm

def calculateBinOSMPolygon_pir_Disk(df, bin_number, mi=20):
    """
    Generates binary relations between types of POIs and the polygonal geographic features using the circunference area (PiR²).
    The binary relations are directly saved in the disk.

    Parameters:
    - df: pandas DataFrame
        The DataFrame containing the data.
    - bin_number: int
        The bin number.
    - mi: int, optional
        The penalty factor (default is 20).

    Returns:
    None
    """
    #Occurrence proportion (OP) and Space proportion (SP) weight
    weights = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
    radius = (1 + bin_number) * 100
    area = math.pi * (radius * radius)
    
    print("executing radius:", radius, "m")

    # File to directly save to disk
    writers = []
    csv_files = []
    for w in weights:
        file_name = './austin-sl-tuple-geoc2vec-' + str(bin_number) + 'bins_polygons_information-wgt' + str(w) + 'pfp-c.csv'
        csv_file = open(file_name, "w", newline='')
        writer = csv.writer(csv_file, delimiter=',')
        writer.writerow(["poi_id_center",
                         "center_poi",
                         "center_poi_level",
                         "context_osm"])
        csv_files.append(csv_file)
        writers.append(writer)

    # Creating communication channel with the database
    try:
        with SSHTunnelForwarder(
            ('localhost', 22),
            ssh_username="root",
            ssh_password="root", 
            remote_bind_address=('localhost', 5432)) as server:

            server.start()
            params = {'database': 'austin_test',
                      'user': 'postgres',
                      'password': 'root',
                      'host': 'localhost',
                      'port': server.local_bind_port
                      }

            connection = psycopg2.connect(**params)

            for id_01, poi in tqdm(df.iterrows(), total=df.shape[0]):
                poi_information = getPOIInformation(connection, poi['business_id'])

                if bin_number == 0:
                    bin_osm_information = getBinOSMInformation(connection, poi['business_id'], bin_number, 'bins_polygons_information')
                    bin_osm_building_information = getBinOSMInformation(connection, poi['business_id'], bin_number, 'bins_polygons_building_information')
                else:
                    bin_osm_information = getBinOSMInformation(connection, poi['business_id'], bin_number, 'continuous_bins_polygons_information')
                    bin_osm_building_information = getBinOSMInformation(connection, poi['business_id'], bin_number, 'continuous_bins_polygons_building_information')

                oc = 0
                if len(bin_osm_information) > 0:
                    tags = list(dict(bin_osm_information[0]).keys())
                    bin_osm_information = pd.DataFrame(bin_osm_information, columns=tags)
                    oc += bin_osm_information.iloc[:, 2:len(tags)-4][~bin_osm_information.iloc[:, 2:len(tags)-4].isin(['None'])].count().sum()
                    tags = tags[2:len(tags)-4]

                if len(bin_osm_building_information) > 0:
                    tags_buildings = list(dict(bin_osm_building_information[0]).keys())
                    bin_osm_building_information = pd.DataFrame(bin_osm_building_information, columns=tags_buildings)
                    oc += bin_osm_building_information.iloc[0]['building_count']
                    tags_buildings = tags_buildings[1:2]

                if oc != 0:
                    if len(bin_osm_information) > 0:
                        for tag in tags:
                            geographic_features = set(bin_osm_information[tag].values)
                            for feature in geographic_features:
                                if feature is not None:
                                    sf = bin_osm_information[bin_osm_information[tag] == feature]['way_area_in'].sum()
                                    of = bin_osm_information[bin_osm_information[tag] == feature][tag].count()
                                    SP = math.ceil((sf / area) * mi)        
                                    OP = math.ceil((of / oc) * mi)
                                    for idx, w in enumerate(weights):
                                        aug = int(math.ceil((w * SP) + ((1 - w) * OP)))
                                        if aug <= 0:
                                            aug = 1
                                        name = "polygons_" + tag + "_" + feature
                                        for center_poi in poi_information:
                                            for b in range(aug):
                                                line = [str(poi['business_id']), 
                                                        str(center_poi['name']),
                                                        str(center_poi['level']),
                                                        str(name)]
                                                writers[idx].writerow(line)

                    if len(bin_osm_building_information) > 0:
                        for id_02, row in bin_osm_building_information.iterrows():
                            sf = row['area_total']
                            of = row['building_count']
                            SP = math.ceil((sf / area) * mi)        
                            OP = math.ceil((of / oc) * mi)
                            for idx, w in enumerate(weights):
                                aug = int(math.ceil((w * SP) + ((1 - w) * OP)))
                                if aug <= 0:
                                    aug = 1
                                name = 'polygons_building_yes'
                                for center_poi in poi_information:
                                    for b in range(aug):
                                        line = [str(poi['business_id']), 
                                                str(center_poi['name']),
                                                str(center_poi['level']),
                                                str(name)]
                                        writers[idx].writerow(line)

        for csv_file in csv_files:
            csv_file.close()
        connection.close()
    
    except Exception as e:
        print(e)

    return None

In [54]:
import csv
import math

def calculateBinOSMRoadsLines_Disk(df, bin_number, mi=20, roads=True):
    """
    Generates binary relations between types of POIs and the linear geographic features.
    The binary relations are directly saved in the disk.

    Parameters:
    - df: pandas DataFrame
        The DataFrame containing the data.
    - bin_number: int
        The bin number.
    - mi: int, optional
        The penalty factor (default is 20).
    - roads: bool, optional
        Flag indicating whether to process roads or lines (default is True).

    Returns:
    None
    """

    # Occurrence proportion (OP) and Space proportion (SP) weight
    weights = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
    radius = (1 + bin_number) * 100
    
    if roads:
        t_name = 'bins_roads_information'
        table = "roads"
        if bin_number == 0:
            materialized_view = 'bins_roads_information'
        else:
            materialized_view = 'continuous_bins_roads_information'
    else:
        t_name = 'bins_lines_information'
        table = "lines"
        if bin_number == 0:
            materialized_view = 'bins_lines_information'
        else:
            materialized_view = 'continuous_bins_lines_information'

    print("executing radius:", radius, "m")

    # Creating files to save directly on the disk
    writers = []
    csv_files = []
    for w in weights:
        file_name = './austin-sl-tuple-geoc2vec-' + str(bin_number) + t_name + '-wgt'+str(w)+'pfp-c.csv'
        csv_file = open(file_name, "w", newline='')
        writer = csv.writer(csv_file, delimiter=',')
        writer.writerow(["poi_id_center",
                          "center_poi",
                          "center_poi_level",
                          "context_osm"])
        csv_files.append(csv_file)
        writers.append(writer)

    # Creating communication channel with the database
    try:
        with SSHTunnelForwarder(
            ('localhost', 22),
            ssh_username="root",
            ssh_password="root", 
            remote_bind_address=('localhost', 5432)) as server:

            server.start()
            params = {'database': 'austin_test',
                      'user': 'postgres',
                      'password': 'root',
                      'host': 'localhost',
                      'port': server.local_bind_port}

            connection = psycopg2.connect(**params)

            for id_01, poi in tqdm(df.iterrows(), total=df.shape[0]):
                poi_information = getPOIInformation(connection, poi['business_id'])
                bin_osm_information = getBinOSMInformation(connection, poi['business_id'], bin_number, materialized_view)

                if len(bin_osm_information) > 0:
                    tags = list(dict(bin_osm_information[0]).keys())
                    bin_osm_information = pd.DataFrame(bin_osm_information, columns=tags)

                    oc = bin_osm_information.iloc[:, 2:len(tags)-3][~bin_osm_information.iloc[:, 2:len(tags)-3].isin(['None'])].count().sum()
                    sc = (bin_osm_information.iloc[:, 2:len(tags)-3][~bin_osm_information.iloc[:, 2:len(tags)-3].isin(['None'])].count(axis=1) * bin_osm_information['length']).sum()

                    tags = tags[2:len(tags)-3]

                    if oc != 0:
                        for tag in tags:
                            geographic_features = set(bin_osm_information[tag].values)
                            for feature in geographic_features:
                                if feature is not None:
                                    
                                    sf = bin_osm_information[bin_osm_information[tag] == feature]['length'].sum()
                                    of = bin_osm_information[bin_osm_information[tag] == feature][tag].count()

                                    SP = math.ceil((sf / sc) * mi)        
                                    OP = math.ceil((of / oc) * mi)
                                    
                                    for idx, w in enumerate(weights):
                                        aug = int(math.ceil((w * SP) + ((1 - w) * OP)))
                                        if aug <= 0:
                                            aug = 1
                                        name = table + "_" + tag + "_" + feature
                                        for center_poi in poi_information: 
                                            for b in range(aug):
                                                line = [str(poi['business_id']), 
                                                        str(center_poi['name']),
                                                        str(center_poi['level']),
                                                        str(name)]
                                                writers[idx].writerow(line)

            for csv_file in csv_files:
                csv_file.close()
            connection.close()
    
    except Exception as e:
        print(e)
        
    return None

In [21]:
import csv
import math

def calculateBinOSMRoadsLines_distance_Disk(df, bin_number, mi=220, roads=True, w=0.5):
    """
    Generates binary relations between types of POIs and the linear geographic features using distance to penalize the relations.
    The binary relations are saved directly to the disk.

    Parameters:
    - df: pandas DataFrame
        The DataFrame containing the data.
    - bin_number: int
        The bin number.
    - mi: int, optional
        The penalty factor (default is 20).
    - roads: bool, optional
        Flag indicating whether to process roads or lines (default is True).
    - w: float, optional
        Weight factor for balancing SP and OP (default is 0.5).

    Returns:
    None
    """

    # Occurrence proportion (OP) and Space proportion (SP) weight
    weights = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
    radius = (1 + bin_number) * 100
    
    w = round(w, 1)
    
    if roads:
        t_name = 'bins_roads_information'
        table = "roads"
        if (bin_number == 0):
            materialized_view = 'bins_roads_information'
        else:
            materialized_view = 'continuous_bins_roads_information'
    else:
        t_name = 'bins_lines_information'
        table = "lines"
        if(bin_number == 0):
            materialized_view = 'bins_lines_information'
        else:
            materialized_view = 'continuous_bins_lines_information'

    print("executing radius:", radius, "m")

    # Creating files to save directly on the disk
    writers = []
    csv_files = []
    for w in weights:
        file_name = './austin-sl-tuple-geoc2vec-' + str(bin_number) + t_name + '-wgt'+str(w)+'pfp-c.csv'
        csv_file = open(file_name, "w", newline='')
        writer = csv.writer(csv_file, delimiter=',')
        writer.writerow(["poi_id_center",
                          "center_poi",
                          "center_poi_level",
                          "context_osm"])
        csv_files.append(csv_file)
        writers.append(writer)

    # Creating communication channel with the database
    try:
        with SSHTunnelForwarder(
            ('localhost', 23456),
            #ssh_private_key="</path/to/private/ssh/key>",
            ### in my case, I used a password instead of a private key
            ssh_username="root",
            ssh_password="root", 
            remote_bind_address=('localhost', 5432)) as server:

                server.start()
                #print("server connected")

                params = {'database': 'austin_test',
                       'user': 'postgres',
                       'password': 'root',
                       'host': 'localhost',
                       'port': server.local_bind_port
                }

                connection = psycopg2.connect(**params)

                for id_01, poi in tqdm(df.iterrows(), total=df.shape[0]):

                    #[business_id, checkin, category]
                    poi_information = getPOIInformation(connection, poi['business_id'])


                    #[business_id, checkin, category, distance_m]
                    bin_osm_information = getBinOSMInformation(connection, poi['business_id'], bin_number, materialized_view)

                    # If the bin is filled with some information
                    if (len(bin_osm_information) > 0):
                        tags = list(dict(bin_osm_information[0]).keys())

                        bin_osm_information = pd.DataFrame(bin_osm_information, columns = tags)

                        # Calculating the two parameters below
                        # oc - total number of roads/lines in the bin
                        oc = bin_osm_information.iloc[:,2:len(tags)-3][~bin_osm_information.iloc[:,2:len(tags)-3].isin(['None'])].count().sum()

                        # sc - total length of each type of roads/lines in the bin
                        sc = (bin_osm_information.iloc[:,2:len(tags)-3][~bin_osm_information.iloc[:,2:len(tags)-3].isin(['None'])].count(axis=1)*bin_osm_information['length']).sum()


                        # Excluding ids and bin_number
                        tags = tags[2:len(tags)-3]

                        # The additions are made based on the labels
                        # To avoid division by zero
                        if(oc != 0):
                            for tag in tags:
                                # Iterating through each tag
                                geographic_features = set(bin_osm_information[tag].values)

                                for feature in geographic_features:

                                    if(feature != None):

                                        # sp = all length of the tag
                                        # op = all occurrences of the tag
                                        sf = bin_osm_information[bin_osm_information[tag] == feature]['length'].sum()
                                        of = bin_osm_information[bin_osm_information[tag] == feature][tag].count()
                                        dst = bin_osm_information[bin_osm_information[tag] == feature]['distance_m'].mean()

                                        SP = math.ceil((sf/sc)*mi)        
                                        OP = math.ceil((of/oc)*mi)
                                        
                                        dst_rel = (dst/radius)

                                        for idx, w in enumerate(weights):
                                            
                                            aug = int(math.ceil(((w*SP) + ((1 - w)*OP))/(1+dst_rel)))
                                            if (aug <= 0):
                                                aug = 1

                                            name = table+"_"+tag+"_"+feature

                                            for center_poi in poi_information: # For each tki
                                                # Increasing it by the factor b
                                                for b in range(aug):
                                                    line = [str(poi['business_id']), 
                                                            str(center_poi['name']),
                                                            str(center_poi['level']),
                                                            str(name)]
                                                    writers[idx].writerow(line)

                                
                for csv_file in csv_files:
                    csv_file.close()
                connection.close()
    
    except Exception as e:
        print(e)
        #print("Connection Failed")
        
    return None

In [35]:
import csv
import math

def calculateBinOSMRoadsLines_Estimated_Disk(df, bin_number, mi=20, roads=True):
    """
    Generates binary relations between types of POIs and the linear geographic features using estimated values for lines/roads lengths.
    The binary relations are saved directly to the disk.

    Parameters:
    - df: pandas DataFrame
        The DataFrame containing the data.
    - bin_number: int
        The bin number.
    - mi: int, optional
        The penalty factor (default is 20).
    - roads: bool, optional
        Flag indicating whether to process roads or lines (default is True).

    Returns:
    None
    """

    radius = (1 + bin_number) * 100

    # Estimated lengths for bins
    lines_values = [738, 1682, 2668, 3297, 4022, 4481, 4984, 6375, 7538]
    roads_values = [347, 699, 1180, 1593, 2010, 2010, 2011, 2435, 2687]

    if roads:
        c = roads_values[bin_number]
        t_name = 'bins_roads_information'
        table = "roads"
        if bin_number == 0:
            materialized_view = 'bins_roads_information'
        else:
            materialized_view = 'continuous_bins_roads_information'
    else:
        c = lines_values[bin_number]
        t_name = 'bins_lines_information'
        table = "lines"
        if bin_number == 0:
            materialized_view = 'bins_lines_information'
        else:
            materialized_view = 'continuous_bins_lines_information'

    print("executing radius:", radius, "m")
    # Occurrence proportion (OP) and Space proportion (SP) weight
    weights = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

    # Creating files to save directly on the disk
    writers = []
    csv_files = []
    for w in weights:
        file_name = './geographic/GEOC2VEC COMP 2/austin-sl-tuple-geoc2vec-' + str(bin_number) + t_name + '-wgt'+str(w)+'pfp-c.csv'
        csv_file = open(file_name, "w", newline='')
        writer = csv.writer(csv_file, delimiter=',')
        writer.writerow(["poi_id_center",
                          "center_poi",
                          "center_poi_level",
                          "context_osm"])
        csv_files.append(csv_file)
        writers.append(writer)

    with SSHTunnelForwarder(
        ('localhost', 23456),
        ssh_username="root",
        ssh_password="root", 
        remote_bind_address=('localhost', 5432)) as server:

            server.start()

            params = {'database': 'austin_test',
                      'user': 'postgres',
                      'password': 'root',
                      'host': 'localhost',
                      'port': server.local_bind_port}

            connection = psycopg2.connect(**params)

            for id_01, poi in tqdm(df.iterrows(), total=df.shape[0]):

                poi_information = getPOIInformation(connection, poi['business_id'])
                bin_osm_information = getBinOSMInformation(connection, poi['business_id'], bin_number, materialized_view)

                # If the bin is filled with some information
                if len(bin_osm_information) > 0:
                    tags = list(dict(bin_osm_information[0]).keys())
                    bin_osm_information = pd.DataFrame(bin_osm_information, columns=tags)

                    # Calculating the two parameters below
                    # oc - total number of roads/lines in the bin
                    oc = bin_osm_information.iloc[:, 2:len(tags)-3][~bin_osm_information.iloc[:, 2:len(tags)-3].isin(['None'])].count().sum()

                    # Excluding ids and bin_number
                    tags = tags[2:len(tags)-3]

                    # Adding based on the labels
                    # To avoid division by zero
                    if oc != 0:
                        for tag in tags:
                            # Iterating through each tag
                            geographic_features = set(bin_osm_information[tag].values)

                            for feature in geographic_features:

                                if feature is not None:

                                    # sp = all length of the tag
                                    # op = all occurrences of the tag
                                    sf = bin_osm_information[bin_osm_information[tag] == feature]['length'].sum()
                                    of = bin_osm_information[bin_osm_information[tag] == feature][tag].count()

                                    # Proportion considering the circumference
                                    SP = math.ceil((sf/c)*mi)
                                    OP = math.ceil((of/oc)*mi)

                                    name = table + "_" + tag + "_" + feature

                                    for idx, w in enumerate(weights):

                                        aug = int(math.ceil((w*SP) + ((1 - w)*OP)))

                                        if aug <= 0:
                                            aug = 1

                                        for center_poi in poi_information: # For each tki
                                            # Increasing it by the factor b
                                            for b in range(aug):
                                                line = [str(poi['business_id']), 
                                                        str(center_poi['name']),
                                                        str(center_poi['level']),
                                                        str(name)]
                                                writers[idx].writerow(line)

            for csv_file in csv_files:
                csv_file.close()
            connection.close()
        
    return None

In [25]:
import csv
import math

def calculateBinOSMPoints_Disk(df, bin_number, mi=20):
    """
    Generates binary relations between types of POIs and the geographic points inside bins.
    The binary relations are saved directly to the disk.

    Parameters:
    - df: pandas DataFrame
        The DataFrame containing the data.
    - bin_number: int
        The bin number.
    - mi: int, optional
        The penalty factor (default is 20).

    Returns:
    None
    """

    radius = (1 + bin_number) * 100
    print("executing radius:", radius)

    # File to save directly to disk
    file_name = './austin-sl-tuple-geoc2vec-' + str(bin_number) + 'bins_points_information-pfp-c.csv'
    
    csv_file = open(file_name, "w", newline='')
    writer = csv.writer(csv_file, delimiter=',')
    writer.writerow(["poi_id_center",
                     "center_poi",
                     "center_poi_level",
                     "context_osm"])

    # Creating communication channel with the database
    connection = connect()  

    if connection:

        for id_01, poi in df.iterrows():
            
            poi_information = getPOIInformation(connection, poi['business_id'])

            if bin_number == 0:
                bin_osm_information = getBinOSMInformation(connection, poi['business_id'], bin_number, 'bins_points_information')
            else:
                bin_osm_information = getBinOSMInformation(connection, poi['business_id'], bin_number, 'continuous_bins_points_information')

            
            # If the bin is filled with some information
            if len(bin_osm_information) > 0:

                tags = list(dict(bin_osm_information[0]).keys())
                bin_osm_information = pd.DataFrame(bin_osm_information, columns=tags)

            
                # oc - total number of different types of points in the bin
                oc = bin_osm_information.iloc[:, 2:len(tags)-2][~bin_osm_information.iloc[:, 2:len(tags)-2].isin(['None'])].count().sum()
                
                # Excluding ids and bin_number
                tags = tags[2:len(tags)-2]
                
                # Adding based on the labels

                # To avoid division by zero
                if oc != 0:

                    for tag in tags:
                        # Iterating through each tag
                        geographic_features = set(bin_osm_information[tag].values)
                        
                        for feature in geographic_features:
                            
                            if feature is not None:
                                
                                of = bin_osm_information[bin_osm_information[tag] == feature][tag].count()
                                
                                OP = (of/oc)*mi
                                
                                aug = math.ceil(OP)
                                if aug <= 0:
                                    aug = 1
                                
                                name = "points_"+tag+"_"+feature


                                for center_poi in poi_information: # For each tki
                                    # Increasing it by the factor b
                                    for b in range(aug):

                                        line = [str(poi['business_id']), 
                                                str(center_poi['name']),
                                                str(center_poi['level']),
                                                str(name)]
                                        writer.writerow(line)
        
        csv_file.close()
        closeConnection(connection)
        
    return None

In [26]:
import csv
import math

def calculateBinOSMPoints_distance_Disk(df, bin_number, mi=20):
    """
    Generates binary relations between types of POIs and the geographic points inside bins,
    taking into account the distance factor. The binary relations are saved directly to the disk.

    Parameters:
    - df: pandas DataFrame
        The DataFrame containing the data.
    - bin_number: int
        The bin number.
    - mi: int, optional
        The penalty factor (default is 20).

    Returns:
    None
    """

    radius = (1 + bin_number) * 100
    print("executing radius:", radius)

    # File to save directly to disk
    file_name = './austin-sl-tuple-geoc2vec-' + str(bin_number) + 'bins_points_information-pfp-c.csv'
    
    csv_file = open(file_name, "w", newline='')
    writer = csv.writer(csv_file, delimiter=',')
    writer.writerow(["poi_id_center",
                     "center_poi",
                     "center_poi_level",
                     "context_osm"])

    # Creating communication channel with the database
    try:

        with SSHTunnelForwarder(
            ('localhost', 23456),
            ssh_username="root",
            ssh_password="root", 
            remote_bind_address=('localhost', 5432)) as server:

                server.start()

                params = {'database': 'austin_test',
                       'user': 'postgres',
                       'password': 'root',
                       'host': 'localhost',
                       'port': server.local_bind_port
                }

                connection = psycopg2.connect(**params)

                for id_01, poi in tqdm(df.iterrows(), total=df.shape[0]):

                    poi_information = getPOIInformation(connection, poi['business_id'])

                    if bin_number == 0:
                        bin_osm_information = getBinOSMInformation(connection, poi['business_id'], bin_number, 'bins_points_information')
                    else:
                        bin_osm_information = getBinOSMInformation(connection, poi['business_id'], bin_number, 'continuous_bins_points_information')


                    if len(bin_osm_information) > 0:

                        tags = list(dict(bin_osm_information[0]).keys())
                        bin_osm_information = pd.DataFrame(bin_osm_information, columns=tags)

                        oc = bin_osm_information.iloc[:, 2:len(tags)-2][~bin_osm_information.iloc[:, 2:len(tags)-2].isin(['None'])].count().sum()

                        tags = tags[2:len(tags)-2]

                        if oc != 0:

                            for tag in tags:
                                geographic_features = set(bin_osm_information[tag].values)

                                for feature in geographic_features:

                                    if feature is not None:

                                        of = bin_osm_information[bin_osm_information[tag] == feature][tag].count()
                                        dst = bin_osm_information[bin_osm_information[tag] == feature]['distance_m'].mean()
                                        OP = (of/oc)*mi
                                        
                                        dst_rel = (dst/radius)

                                        aug = math.ceil(OP/(1+dst_rel))
                                        
                                        if aug <= 0:
                                            aug = 1
                                            

                                        name = "points_"+tag+"_"+feature

                                        for center_poi in poi_information: 
                                            for b in range(aug):

                                                line = [str(poi['business_id']), 
                                                        str(center_poi['name']),
                                                        str(center_poi['level']),
                                                        str(name)]
                                                writer.writerow(line)

            
                connection.close()
        
    
    except Exception as e:
        print(e)
        
    csv_file.close()
        
    return None

# Generating Training Data

In [27]:
pois_file_name = './austin-ml-updated.csv'
df = pd.read_csv(pois_file_name)
print(df.shape)
df = df.dropna()
print(df.shape)
df.head()

(22399, 7)
(22399, 7)


,business_id,city,state,latitude,longitude,categories,checkin_count
0,N3_Gs3DnX4k9SgpwJxdEfw,Austin,TX,30.346169,-97.711458,"Shopping, Jewelry Repair, Appraisal Services, ...",14
1,tXvdYGvlEceDljN8gt2_3Q,Austin,TX,30.172706,-97.799920,"Barbers, Beauty & Spas",1
2,nTIhpR7MhsALPwg_Hh14EA,Austin,TX,30.326377,-97.704543,"Hotels, Hotels & Travel, Event Planning & Serv...",475
3,8XyEpVdAO0o6iVkVxkWosQ,Austin,TX,30.246465,-97.778738,"Home Services, Real Estate, Property Management",0
4,NVfOn7TdnHbaGH97CVB_Qg,Austin,TX,30.244902,-97.857409,"Chiropractors, Health & Medical",33


## Iteractive Execution

In [ ]:
#Execution of functions that generate the training set for GeoContext2Vec using relative space proportions.
for bin_number in range(0, 2):
    calculateBinOSMPolygon_Disco(df, bin_number)
    calculateBinOSMRoadsLines_Disco(df, bin_number, roads=False)
    calculateBinOSMRoadsLines_Disco(df, bin_number, roads=True)
    calculateBinOSMPoints_Disco(df, n)

In [ ]:
#Execution of functions that generate the training set for GeoContext2Vec using absolute space proportions.
for bin_number in range(0, 2):
    calculateBinOSMPolygon_pir_Disk(df, bin_number)
    calculateBinOSMRoadsLines_Estimated_Disk(df, bin_number, roads=False)
    calculateBinOSMRoadsLines_Estimated_Disk(df, bin_number, roads=True)
    calculateBinOSMPoints_Disco(df, n)

## Parallel Execution Example

In [21]:
import multiprocessing as mp
print("Number of processors: ", mp.cpu_count())

# Step 1: Init multiprocessing.Pool()
pool = mp.Pool(int(mp.cpu_count()))

# Step 2: `pool.apply` the `howmany_within_range()`
bins = [0, 1, 2, 3, 4, 5, 6, 7, 8]
mi = 20

pool.starmap(calculateBinOSMPolygon_distance_Disk, [(df, n,  mi) for n in bins])
pool.starmap(calculateBinOSMRoadsLines_distance_Disk, [(df, n,  mi, True) for n in bins])
pool.starmap(calculateBinOSMRoadsLines_distance_Disk, [(df, n,  mi, False) for n in bins])
pool.starmap(calculateBinOSMPoints_dst_Disk, [(df, n,  mi) for n in bins])


# Step 3: Don't forget to close
pool.close()

Number of processors:  20
executing radius: 100 m
executing radius: 200 m
executing radius: 300 m
executing radius: 400 m
executing radius: 500 m
executing radius: 600 m
executing radius: 700 m
executing radius: 800 m
executing radius: 900 m


100%|██████████| 22399/22399 [30:12<00:00, 12.36it/s]


executing radius: 300 m
executing radius: 400 m
executing radius: 500 m
executing radius: 100 m
executing radius: 600 m
executing radius: 700 m
executing radius: 200 m
executing radius: 800 m
executing radius: 900 m


100%|██████████| 22399/22399 [21:04<00:00, 17.71it/s]


executing radius: 100 m
executing radius: 200 m
executing radius: 300 m
executing radius: 400 m
executing radius: 500 m
executing radius: 600 m
executing radius: 700 m
executing radius: 800 m
executing radius: 900 m


100%|██████████| 22399/22399 [37:55<00:00,  9.84it/s]


executing radius: 100
executing radius: 200
executing radius: 300
executing radius: 400
executing radius: 500
executing radius: 600
executing radius: 700
executing radius: 800
executing radius: 900


 85%|████████▍ | 19001/22399 [17:55<03:08, 18.06it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

100%|██████████| 22399/22399 [24:21<00:00, 15.33it/s]
